# Исследование данных об инвестиции венчурных фондов в компании-стартапы

## Данные

Самостоятельная работа основана на базе данных, которая хранит информацию о венчурных фондах и инвестициях в компании-стартапы, и основана на датасете Startup Investments https://www.kaggle.com/justinas/startup-investments.

## Цель

Проанализировать данные о фондах и инвестициях, произвести выгрузки данных и ответить на поставленные вопросы с помощью SQL.


### Задание 1
Посчитайте, сколько компаний закрылось.


In [ ]:
'''
SELECT COUNT(id)
FROM company
WHERE status = 'closed';
'''

### Задание 2
Отобразите количество привлечённых средств для новостных компаний США.
Используйте данные из таблицы company. Отсортируйте таблицу по убыванию значений в поле funding_total.



In [ ]:
'''
SELECT SUM(funding_total)
FROM company
WHERE category_code = 'news'
      AND country_code = 'USA'
GROUP BY id
ORDER BY funding_total DESC;
'''

### Задание 3
Найдите общую сумму сделок по покупке одних компаний другими в долларах. Отберите сделки, которые осуществлялись только за наличные с 2011 по 2013 год включительно.



In [ ]:
'''
SELECT SUM(price_amount)
FROM acquisition
WHERE term_code = 'cash'
      AND CAST(acquired_at AS date) BETWEEN '2011-01-01' AND '2013-12-31';
'''

### Задание 4
Отобразите имя, фамилию и названия аккаунтов людей в твиттере, у которых названия аккаунтов начинаются на 'Silver'.



In [ ]:
'''
SELECT first_name,
       last_name,
       twitter_username
FROM people
WHERE twitter_username LIKE 'Silver%';
'''

### Задание 5
Выведите на экран всю информацию о людях, у которых названия аккаунтов в твиттере содержат подстроку 'money', а фамилия начинается на 'K'



In [ ]:
'''
SELECT *
FROM people
WHERE twitter_username LIKE '%money%'
      AND last_name LIKE 'K%';
'''

### Задание 6
Для каждой страны отобразите общую сумму привлечённых инвестиций, которые получили компании, зарегистрированные в этой стране. Страну, в которой зарегистрирована компания, можно определить по коду страны. Отсортируйте данные по убыванию суммы.



In [ ]:
'''
SELECT country_code,
       SUM(funding_total)
FROM company
GROUP BY country_code
ORDER BY SUM(funding_total) DESC;
'''

### Задание 7
Составьте таблицу, в которую войдёт дата проведения раунда, а также минимальное и максимальное значения суммы инвестиций, привлечённых в эту дату. Оставьте в итоговой таблице только те записи, в которых минимальное значение суммы инвестиций не равно нулю и не равно максимальному значению.


In [ ]:
'''
SELECT funded_at,
       MIN(raised_amount),
       MAX(raised_amount)
FROM funding_round
GROUP BY funded_at
HAVING MIN(raised_amount) <> 0
      AND MIN(raised_amount) <> MAX(raised_amount);
'''

### Задание 8
Создайте поле с категориями:

Для фондов, которые инвестируют в 100 и более компаний, назначьте категорию high_activity.

Для фондов, которые инвестируют в 20 и более компаний до 100, назначьте категорию middle_activity.

Если количество инвестируемых компаний фонда не достигает 20, назначьте категорию low_activity.

Отобразите все поля таблицы fund и новое поле с категориями.


In [ ]:
'''
SELECT *,
       CASE
           WHEN invested_companies >= 100 THEN 'high_activity'
           WHEN invested_companies >= 20 AND invested_companies < 100 THEN 'middle_activity'
           WHEN invested_companies < 20 THEN 'low_activity'
       END
FROM fund;
'''

### Задание 9
Для каждой из категорий, назначенных в предыдущем задании, посчитайте округлённое до ближайшего целого числа среднее количество инвестиционных раундов, в которых фонд принимал участие. Выведите на экран категории и среднее число инвестиционных раундов. Отсортируйте таблицу по возрастанию среднего.

In [ ]:
'''
SELECT ROUND(AVG(investment_rounds)),
       CASE
           WHEN invested_companies>=100 THEN 'high_activity'
           WHEN invested_companies>=20 THEN 'middle_activity'
           ELSE 'low_activity'
       END AS activity
FROM fund
GROUP BY activity
ORDER BY ROUND(AVG(investment_rounds));
'''

### Задание 10

Проанализируйте, в каких странах находятся фонды, которые чаще всего инвестируют в стартапы.
Для каждой страны посчитайте минимальное, максимальное и среднее число компаний, в которые инвестировали фонды этой страны, основанные с 2010 по 2012 год включительно. Исключите страны с фондами, у которых минимальное число компаний, получивших инвестиции, равно нулю.
Выгрузите десять самых активных стран-инвесторов: отсортируйте таблицу по среднему количеству компаний от большего к меньшему. Затем добавьте сортировку по коду страны в лексикографическом порядке.


In [ ]:
'''
SELECT country_code,
       MIN(invested_companies),
       MAX(invested_companies),
       AVG(invested_companies)
FROM fund
WHERE CAST(founded_at AS date) BETWEEN '2010-01-01' AND '2012-12-31'
GROUP BY country_code
HAVING MIN(invested_companies) <> 0
ORDER BY AVG(invested_companies) DESC, country_code
LIMIT 10;
'''

### Задание 11
Отобразите имя и фамилию всех сотрудников стартапов. Добавьте поле с названием учебного заведения, которое окончил сотрудник, если эта информация известна.



In [ ]:
'''
SELECT p.first_name,
       p.last_name,
       e.instituition
FROM people AS p
LEFT JOIN education AS e ON p.id = e.person_id;
'''

### Задание 12
Для каждой компании найдите количество учебных заведений, которые окончили её сотрудники. Выведите название компании и число уникальных названий учебных заведений. Составьте топ-5 компаний по количеству университетов.



In [ ]:
'''
SELECT c.name,
       COUNT(DISTINCT e.instituition)
FROM company AS c
JOIN people AS p ON c.id = p.company_id
JOIN education AS e ON p.id = e.person_id
GROUP BY c.name
ORDER BY COUNT(DISTINCT e.instituition) DESC
LIMIT 5;
'''

### Задание 13
Составьте список с уникальными названиями закрытых компаний, для которых первый раунд финансирования оказался последним.



In [ ]:
'''
SELECT DISTINCT c.name
FROM company AS c
JOIN funding_round AS f ON c.id = f.company_id
WHERE c.status = 'closed'
      AND f.is_first_round = 1
      AND f.is_last_round = 1;
'''



### Задание 14
Составьте список уникальных номеров сотрудников, которые работают в компаниях, отобранных в предыдущем задании.



In [ ]:
'''
SELECT DISTINCT p.id
FROM people AS p
WHERE p.company_id IN
(SELECT DISTINCT c.id
FROM company AS c
JOIN funding_round AS f ON c.id = f.company_id
WHERE c.status = 'closed'
      AND f.is_first_round = 1
      AND f.is_last_round = 1);
'''

### Задание 15
Составьте таблицу, куда войдут уникальные пары с номерами сотрудников из предыдущей задачи и учебным заведением, которое окончил сотрудник.



In [ ]:
'''
SELECT DISTINCT p.id,
       e.instituition
FROM education AS e
JOIN people AS p ON e.person_id = p.id
WHERE e.person_id IN
(SELECT DISTINCT p.id
FROM people AS p
WHERE p.company_id IN
(SELECT DISTINCT c.id
FROM company AS c
JOIN funding_round AS f ON c.id = f.company_id
WHERE c.status = 'closed'
      AND f.is_first_round = 1
      AND f.is_last_round = 1));
'''

### Задание 16
Посчитайте количество учебных заведений для каждого сотрудника из предыдущего задания. При подсчёте учитывайте, что некоторые сотрудники могли окончить одно и то же заведение дважды.



In [ ]:
'''
SELECT p.id,
       COUNT(e.instituition)
FROM education AS e
JOIN people AS p ON e.person_id = p.id
WHERE e.person_id IN
(SELECT DISTINCT p.id
FROM people AS p
WHERE p.company_id IN
(SELECT DISTINCT c.id
FROM company AS c
JOIN funding_round AS f ON c.id = f.company_id
WHERE c.status = 'closed'
      AND f.is_first_round = 1
      AND f.is_last_round = 1))
GROUP BY p.id;
'''

### Задание 17

Дополните предыдущий запрос и выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники разных компаний. Нужно вывести только одну запись, группировка здесь не понадобится.



In [ ]:
'''
WITH
avg_1 AS
(SELECT p.id,
       COUNT(e.instituition)
FROM education AS e
JOIN people AS p ON e.person_id = p.id
WHERE e.person_id IN
(SELECT DISTINCT p.id
FROM people AS p
WHERE p.company_id IN
(SELECT DISTINCT c.id
FROM company AS c
JOIN funding_round AS f ON c.id = f.company_id
WHERE c.status = 'closed'
      AND f.is_first_round = 1
      AND f.is_last_round = 1))
GROUP BY p.id)
SELECT AVG(COUNT)
FROM avg_1;
'''

### Задание 18
Напишите похожий запрос: выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники Facebook*.
*(сервис, запрещённый на территории РФ)



In [ ]:
'''
WITH
avg_2 AS
(SELECT p.id,
       COUNT(e.instituition)
FROM education AS e
JOIN people AS p ON e.person_id = p.id
WHERE e.person_id IN
(SELECT DISTINCT p.id
FROM people AS p
WHERE p.company_id IN
(SELECT DISTINCT c.id
FROM company AS c
JOIN funding_round AS fr ON c.id = fr.company_id
WHERE c.name = 'Facebook'))
GROUP BY p.id)
SELECT AVG(COUNT)
FROM avg_2;
'''



### Задание 19
Составьте таблицу из полей:
- name_of_fund — название фонда;
- name_of_company — название компании;
- amount — сумма инвестиций, которую привлекла компания в раунде.

В таблицу войдут данные о компаниях, в истории которых было больше шести важных этапов, а раунды финансирования проходили с 2012 по 2013 год включительно.



In [ ]:
'''
SELECT f.name AS name_of_fund,
       c.name AS name_of_company,
       fr.raised_amount AS amount
FROM company AS c
JOIN funding_round AS fr ON c.id = fr.company_id
JOIN investment AS i ON fr.id = i.funding_round_id
JOIN fund AS f ON i.fund_id = f.id
WHERE c.milestones > 6
      AND CAST(fr.funded_at AS date) BETWEEN '2012.01.01' AND '2013.12.31';
'''

### Задание 20

Выгрузите таблицу, в которой будут такие поля:
- название компании-покупателя;
- сумма сделки;
- название компании, которую купили;
- сумма инвестиций, вложенных в купленную компанию;
- доля, которая отображает, во сколько раз сумма покупки превысила сумму вложенных в компанию инвестиций, округлённая до ближайшего целого числа.

Не учитывайте те сделки, в которых сумма покупки равна нулю. Если сумма инвестиций в компанию равна нулю, исключите такую компанию из таблицы.

Отсортируйте таблицу по сумме сделки от большей к меньшей, а затем по названию купленной компании в лексикографическом порядке. Ограничьте таблицу первыми десятью записями.



In [ ]:
'''
SELECT c1.name AS buyer,
       a.price_amount AS price,
       c2.name AS seller,
       c2.funding_total AS funding,
       ROUND(a.price_amount/c2.funding_total) AS proportion
FROM acquisition AS a
INNER JOIN company AS c1 ON a.acquiring_company_id = c1.id
INNER JOIN company AS c2 ON a.acquired_company_id = c2.id
WHERE a.price_amount != 0 AND c2.funding_total != 0
ORDER BY price DESC, seller
LIMIT 10;
'''

### Задание 21

Выгрузите таблицу, в которую войдут названия компаний из категории social, получившие финансирование с 2010 по 2013 год включительно. Проверьте, что сумма инвестиций не равна нулю. Выведите также номер месяца, в котором проходил раунд финансирования.



In [ ]:
'''
SELECT c.name,
       EXTRACT(MONTH FROM CAST(fr.funded_at AS date))
FROM company AS c
JOIN funding_round AS fr ON c.id = fr.company_id
WHERE c.category_code = 'social'
      AND CAST(fr.funded_at AS date) BETWEEN '2010-01-01' AND '2013-12-31'
      AND fr.raised_amount <> 0;
'''

### Задание 22
Отберите данные по месяцам с 2010 по 2013 год, когда проходили инвестиционные раунды. Сгруппируйте данные по номеру месяца и получите таблицу, в которой будут поля:
- номер месяца, в котором проходили раунды;
- количество уникальных названий фондов из США, которые инвестировали в этом месяце;
- количество компаний, купленных за этот месяц;
- общая сумма сделок по покупкам в этом месяце.


In [ ]:
'''
WITH
fundings AS
(SELECT EXTRACT(MONTH FROM CAST(fr.funded_at AS DATE)) AS funding_month,
COUNT(DISTINCT f.id) AS us_funds
FROM fund AS f
LEFT JOIN investment AS i ON f.id = i.fund_id
LEFT JOIN funding_round AS fr ON i.funding_round_id = fr.id
WHERE f.country_code = 'USA'
AND EXTRACT(YEAR FROM CAST(fr.funded_at AS DATE)) BETWEEN 2010 AND 2013
GROUP BY funding_month),
acquisitions AS
(SELECT EXTRACT(MONTH FROM CAST(acquired_at AS DATE)) AS funding_month,
COUNT(acquired_company_id) AS bought_co,
SUM(price_amount) AS sum_total
FROM acquisition
WHERE EXTRACT(YEAR FROM CAST(acquired_at AS DATE)) BETWEEN 2010 AND 2013
GROUP BY funding_month)
SELECT fnd.funding_month, fnd.us_funds, acq.bought_co, acq.sum_total
FROM fundings AS fnd
LEFT JOIN acquisitions AS acq ON fnd.funding_month = acq.funding_month;

'''

### Задание 23
Составьте сводную таблицу и выведите среднюю сумму инвестиций для стран, в которых есть стартапы, зарегистрированные в 2011, 2012 и 2013 годах. Данные за каждый год должны быть в отдельном поле. Отсортируйте таблицу по среднему значению инвестиций за 2011 год от большего к меньшему.


In [ ]:

'''
WITH y_11 AS
(SELECT country_code AS country,
AVG(funding_total) AS y_2011
FROM company
WHERE EXTRACT(YEAR FROM founded_at::DATE) IN(2011, 2012, 2013)
GROUP BY country, EXTRACT(YEAR FROM founded_at)
HAVING EXTRACT(YEAR FROM founded_at) = '2011'),
y_12 AS
(SELECT country_code AS country,
AVG(funding_total) AS y_2012
FROM company
WHERE EXTRACT(YEAR FROM founded_at::DATE) IN(2011, 2012, 2013)
GROUP BY country, EXTRACT(YEAR FROM founded_at)
HAVING EXTRACT(YEAR FROM founded_at) = '2012'),
y_13 AS
(SELECT country_code AS country,
AVG(funding_total) AS y_2013
FROM company
WHERE EXTRACT(YEAR FROM founded_at::DATE) IN(2011, 2012, 2013)
GROUP BY country, EXTRACT(YEAR FROM founded_at)
HAVING EXTRACT(YEAR FROM founded_at) = '2013')
SELECT y_11.country, y_2011, y_2012, y_2013
FROM y_11
JOIN y_12 ON y_11.country = y_12.country
JOIN y_13 ON y_12.country = y_13.country
ORDER BY y_2011 DESC;
'''